# mcmc

> Fill in a module description here

In [ ]:
#| default_exp mcmc

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import numpy as np
import emcee 
import time
from CubicGalileonEmu.emu import emu_redshift

In [ ]:
#| export

def ln_prior(theta, params_list):
    
    p1, p2, p3, p4, p5, _ = theta
    param1, param2, param3, param4, param5 = params_list
    
    if param1[2] < p1 < param1[3]  \
    and param2[2] < p2 < param2[3] \
    and param3[2] < p3 < param3[3] \
    and param4[2] < p4 < param4[3] \
    and param5[2] < p5 < param5[3]:
    
        ### gaussian prior on a
        p1_mu = 0.5*(param1[3] - param1[2]) + param1[2]
        p2_mu = 0.5*(param2[3] - param2[2]) + param2[2]
        p3_mu = 0.5*(param3[3] - param3[2]) + param3[2]
        p4_mu = 0.5*(param4[3] - param4[2]) + param4[2]
        p5_mu = 0.5*(param5[3] - param5[2]) + param5[2]

        p1_sigma = 1*(param1[3] - p1_mu)
        p2_sigma = 1*(param2[3] - p2_mu)
        p3_sigma = 1*(param3[3] - p3_mu)
        p4_sigma = 1*(param4[3] - p4_mu)
        p5_sigma = 1*(param5[3] - p5_mu)
        

        pdf1 = np.log(1.0/(np.sqrt(2*np.pi)*p1_sigma))-0.5*(p1-p1_mu)**2/p1_sigma**2
        pdf2 = np.log(1.0/(np.sqrt(2*np.pi)*p2_sigma))-0.5*(p2-p2_mu)**2/p2_sigma**2
        pdf3 = np.log(1.0/(np.sqrt(2*np.pi)*p3_sigma))-0.5*(p3-p3_mu)**2/p3_sigma**2
        pdf4 = np.log(1.0/(np.sqrt(2*np.pi)*p4_sigma))-0.5*(p4-p4_mu)**2/p4_sigma**2
        pdf5 = np.log(1.0/(np.sqrt(2*np.pi)*p5_sigma))-0.5*(p5-p5_mu)**2/p5_sigma**2
    
        return pdf1 + pdf2 + pdf3 + pdf4 + pdf5

    return -np.inf

In [ ]:
#| export

def ln_like(theta, x_grid, sepia_model_list, z_all, x, y, yerr):
      
    p1, p2, p3, p4, p5, p6 = theta
    
    new_params = np.array([p1, p2, p3, p4, p5, p6])[np.newaxis, :]
        
    model_grid, model_var_grid = emu_redshift(new_params, sepia_model_list, z_all)
        
    model = np.interp(x, x_grid, model_grid[:, 0])
    model_var = np.interp(x, x_grid, model_var_grid[:, 0, 0])
  
    sigma2 = yerr**2  + model_var
    ll = -0.5 * np.sum((y - model)** 2 / sigma2 )
    
    return ll


In [ ]:
#| export

def ln_prob(theta, params_list, x_grid, sepia_model_list, z_all, x, y, yerr):
    
    lp = ln_prior(theta, params_list)
    if not np.isfinite(lp):
        return -np.inf
    return lp + ln_like(theta, x_grid, sepia_model_list, z_all, x, y, yerr)

In [ ]:
#| export

def chain_init(params_list, ndim, nwalkers):

    param1, param2, param3, param4, param5 = params_list

    pos0 = [[param1[1]*1.0, 
               param2[1]*1.0, 
               param3[1]*1.0, 
               param4[1]*1.0,
               param5[1]*1.0] 
            + 1e-3*np.random.randn(ndim + 1) for i in range(nwalkers)]
    
    return pos0


In [ ]:
#| export

def mcmc_run(): pass

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()